In [2]:
import pandas as pd
from difflib import SequenceMatcher
import random
import re

In [2]:
# read files
df = pd.read_csv('df_centuries.csv')
pd.set_option('display.max_columns', None)
#df = df.head()

In [3]:
df_17th = df[df['century'] == '1600s']
df_18th = df[df['century'] == '1700s']
df_19th = df[df['century'] == '1800s']
df_20th = df[df['century'] == '1900s']

In [4]:
samples_per_dataset = 25

In [5]:
selections = []
n = samples_per_dataset #samples per dataset

for dataset in [df_17th, df_18th, df_19th, df_20th]:
    docs = dataset.sample(n)
    docs['dataset'] = f'{dataset}'
    selections.append(docs)

sample = pd.concat(selections)

In [6]:

characters = 1000

for i in range(len(selections)):
    selections[i] = selections[i].reset_index(drop=True)
    for j in range(len(selections[i])):
        if len(selections[i]['gt text'][j]) > characters:
            first_page = selections[i]['gt text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['gt text'][j] = first_page
        if len(selections[i]['ocr text'][j]) > characters:
            first_page = selections[i]['ocr text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['ocr text'][j] = first_page

<ipython-input-6-d281e66b9636>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selections[i]['gt text'][j] = first_page
<ipython-input-6-d281e66b9636>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selections[i]['ocr text'][j] = first_page


In [7]:
def preprocessing(text):
    text = re.sub("[^a-zA-Z-. ]+", '', text) # remove punctuation except for hyphens and dots
    text = text.lower() # lowercase
    return text

for i in range(len(selections)):
    # preprocess OCR and GT
    selections[i]['gt text'] = selections[i]['gt text'].apply(lambda x: preprocessing(x))
    selections[i]['ocr text'] = selections[i]['ocr text'].apply(lambda x: preprocessing(x))
    

In [8]:
def fuzzy_matching(row, samples_per_dataset): # create files with matched sentences

# split GT and OCR-output of each row into sentences (based on full stops).
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")
    
    match_list = []
    
    for sentence in gt_sentences:
        for match in ocr_sentences:
            if len(sentence) >= 4 and len(match) >= 4: 
                if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                    match_list.extend([[sentence, match]])
    
    print('len match_list:', len(match_list))
    
    matched_gt_sentences = []
    matched_ocr_sentences = []
    
    for sentence in match_list:
        matched_gt_sentences.append(sentence[0])
        matched_ocr_sentences.append(sentence[1])

            
    if len(match_list) > 0: # check if there are enough sentences to select randomly
    #    #print('sentlong:', matched_gt_sentences)
        rannum = random.randint(0,len(matched_gt_sentences)-1)
        matched_gt_sentence = matched_gt_sentences[rannum]
        matched_ocr_sentence = matched_ocr_sentences[rannum]
    else: # otherwise note a sentence is missing
        matched_gt_sentence, matched_ocr_sentence = 'None', 'None'
        #print('none')
        
    return matched_gt_sentence, matched_ocr_sentence
    
        
GT_sentences_datasets = []        
OCR_sentences_datasets = []  
samples_per_dataset = samples_per_dataset
    
for dataset in selections:
    dataset.reset_index(drop = True)
    print('DATASET new')
    GT_matched_sentences_list = []
    OCR_matched_sentences_list = []
    counter = 0
    
    i = 0
    while True:
        #print('len:', len(GT_matched_sentences_list))
        if i == samples_per_dataset:
            print('happened')
            i = 0

        # match sentences GT and OCR and create new documents
        #GT_matched_sentences_list = []
        #OCR_matched_sentences_list = []
        print('i', i)
        #print(dataset.iloc[i])
        
        
        matched_gt_sentence, matched_ocr_sentence = fuzzy_matching(dataset.iloc[i], samples_per_dataset)
        #print('found:', matched_gt_sentence, matched_ocr_sentence)
        if matched_gt_sentence != 'None' and (matched_gt_sentence not in GT_matched_sentences_list):
            print('appended')
            counter += 1
            GT_matched_sentences_list.append(matched_gt_sentence)
            OCR_matched_sentences_list.append(matched_ocr_sentence)
        if counter == samples_per_dataset:
            break
        i += 1
        #print('i again', i)
        print('PASSED')

    #print('sampled sentences:', len(GT_matched_sentences_list))
    GT_sentences_datasets.append(GT_matched_sentences_list)
    OCR_sentences_datasets.append(OCR_matched_sentences_list)

DATASET new
i 0
len match_list: 1
appended
PASSED
i 1
len match_list: 1
appended
PASSED
i 2
len match_list: 2
appended
PASSED
i 3
len match_list: 1
appended
PASSED
i 4
len match_list: 1
appended
PASSED
i 5
len match_list: 1
PASSED
i 6
len match_list: 1
appended
PASSED
i 7
len match_list: 6
appended
PASSED
i 8
len match_list: 1
appended
PASSED
i 9
len match_list: 2
appended
PASSED
i 10
len match_list: 1
appended
PASSED
i 11
len match_list: 1
appended
PASSED
i 12
len match_list: 1
appended
PASSED
i 13
len match_list: 2
PASSED
i 14
len match_list: 3
appended
PASSED
i 15
len match_list: 5
appended
PASSED
i 16
len match_list: 3
PASSED
i 17
len match_list: 2
PASSED
i 18
len match_list: 10
appended
PASSED
i 19
len match_list: 3
appended
PASSED
i 20
len match_list: 4
appended
PASSED
i 21
len match_list: 1
appended
PASSED
i 22
len match_list: 6
appended
PASSED
i 23
len match_list: 1
appended
PASSED
i 24
len match_list: 1
appended
PASSED
happened
i 0
len match_list: 1
PASSED
i 1
len match_list: 

In [9]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == '' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,10):
                        if i-j >= 0:
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i-j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i+j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            try:
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                    pass
            except:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before forwards match GT:', GT)
                #print('before forwards bad match OCR:', OCR)
                if i-1 >= 0:
                    GT.insert(i, '')
                else:
                    GT.insert(0, '')
                if i+1 <= len(GT):
                    OCR.insert(i+1, '')
                else:
                    GT.append('')
                #print('after forwards match GT:', GT)
                #print('after forwards bad match OCR:', OCR)
                
                continue
    
   #print('after forwards GT:', GT)
    #print('after forwards OCR:', OCR)
    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')
    
    if True:
        
        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
            #print('i:', i)
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.65:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,len(GT)):
                            if j >= 10:
                                extra_ratio = 0.1
                            else:
                                extra_ratio = 0
                            if i-j >= -len(OCR):
                                if j >= 5:
                                    if (len(GT[i]) >= 5) or (len(OCR[i-j]) >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i-j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    if i+k < 0:
                                        GT.insert(i+1, '')
                                    else: 
                                        GT.append('')
                                break
                            if i+j <= -1:
                                if j >= 5:
                                    if (GT[i] >= 5) or (OCR[i+j] >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i+j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.30:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before backwards bad match GT:',  GT)
                #print('before backwards bad match OCR:',  OCR)
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    GT.insert(i, '')
                    OCR.insert(i+1, '')
                #print('after backwards bad match GT:',  GT)
                #print('after backwards bad match OCR:',  OCR)
                
                
    #print('after backwards GT:',  GT)
    #print('after backwards OCR:',  OCR)
                    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')


    for i in range(len(GT)-1):
        try:
            #print(i)
            #print(GT[i], OCR[i])
            if (GT[i] == '') and (OCR[i] == ''):
                del GT[i]
                del OCR[i]
        except:
            break
            
    try:
        if SequenceMatcher(None, GT[-1], OCR[-2]).ratio() >= 0.65 and ((GT[-1] != '') or ((OCR[-2] != ''))):
            #print('FOUND:', GT[-1], OCR[-2])
            OCR.insert(-2, '')
            GT.append('')
    except:
        pass
    try:
        if SequenceMatcher(None, GT[-2], OCR[-1]).ratio() >= 0.65 and ((GT[-2] != '') or ((OCR[-1] != ''))):
            #print('FOUND:', OCR[-1], GT[-2])
            GT.insert(-2, '')
            OCR.append('')
    except:
        pass
    
    for i in range(len(OCR)):
        if GT[i] == '' and OCR[i] == '':
            GT[i] = 'REMOVE'
            OCR[i] = 'REMOVE'

    GT = list(filter(lambda a: a != 'REMOVE', GT))
    OCR = list(filter(lambda a: a != 'REMOVE', OCR))
    
    
    #print('end GT:',  GT)
    #print('end OCR:',  OCR)
    return GT, OCR



In [10]:
# word alignment
aligned_datasets_GT = []
aligned_datasets_OCR = []

for i in range(len(GT_sentences_datasets)): # for each dataset
    aligned_dataset_GT = []
    aligned_dataset_OCR = []
    for j in range(len(GT_sentences_datasets[i])): # for each sampled sentences in the dataset
        #print('GT_sentences_datasets[i][j]:', GT_sentences_datasets[i][j])
        aligned_GT, aligned_OCR = word_alignment(GT_sentences_datasets[i][j].split(' '), OCR_sentences_datasets[i][j].split(' '))
        aligned_dataset_GT.append(aligned_GT)
        aligned_dataset_OCR.append(aligned_OCR) 
    aligned_datasets_GT.append(aligned_dataset_GT)
    aligned_datasets_OCR.append(aligned_dataset_OCR)

print(len(aligned_datasets_GT))
print(len(aligned_datasets_GT[i]))

4
25


In [11]:
# verify word alignment quality
def verify_alignment(GT, OCR):
    
    if len(GT) != len(OCR):
        alignment_validation = False
        return alignment_validation
    
    min_word_length = 8 # the higher this number, the more likely a alignment will be seen as valid, thus higher chance at false positive, lower chance at false negative. 
    match_ratio = 0.70
    anchors_list = []
    for i in range(len(GT)):
        if len(GT[i]) >= min_word_length:  
            anchors_list.append(GT[i])
        if len(OCR[i]) >= min_word_length:
            anchors_list.append(OCR[i])
    #print(anchors_list)


    alignment_validation = True

    for anchor in anchors_list:
        #print('ANCHOR:', anchor)
        matches_GT = set()
        matches_OCR = set()
        match = False
        for i in range(len(GT)):
            if (SequenceMatcher(None, GT[i], anchor).ratio() >= 0.65) and (len(GT[i]) >= min_word_length or len(GT[i]) == 0):
                matches_GT.add(i)
                #print('added as GT match:', GT[i])
            if SequenceMatcher(None, OCR[i], anchor).ratio() >= 0.65 and (len(OCR[i]) >= min_word_length or len(OCR[i]) == 0):
                matches_OCR.add(i)
                #print('added as OCR match:', OCR[i])
        #print('anchor:', anchor)
        #print(matches_GT)
        #print(matches_OCR)
        if matches_GT == matches_OCR:
            match = True
        if match == False:
            match = True
            points_set1 = set() # counts when the matched opposite word is '' for OCR
            points_set2 = set() # counts when the matched opposite word is '' for GT
            for index in matches_GT.difference(matches_OCR): # elements in matches_GT that are not in matches_OCR
                if OCR[index] == '':
                    points_set1.add(index)
                if OCR[index] != '' and len(matches_OCR.difference(matches_GT)) != 0:
                    match = False
            for index in matches_OCR.difference(matches_GT): # elements in matches_OCR that are not in matches_GT
                if GT[index] == '':
                    points_set2.add(index)
                if GT[index] != '' and len(matches_GT.difference(matches_OCR)) != 0:
                    match = False
            if len(points_set1.intersection(points_set2)) != 0: # if an anchor words was match in both OCR and GT with an '', it's a bad match.
                match = False
        if match == False:
            for index in (matches_GT.difference(matches_OCR)):
                index += 1
                if index in matches_OCR.difference(matches_GT):
                    if (GT[index-1] == '' or OCR[index-1] == '') and (SequenceMatcher(None, GT[index], OCR[index]).ratio() >= match_ratio):
                        match = True
            for index in (matches_OCR.difference(matches_GT)):
                index += 1
                if index in matches_GT.difference(matches_OCR):
                    if (GT[index] == '' or OCR[index] == '') and (SequenceMatcher(None, GT[index-1], OCR[index-1]).ratio() >= match_ratio):
                        match = True

        if match == False:
            alignment_validation = False

    return alignment_validation

In [12]:
# verify word alignment
good_alignments_datasets = []
bad_alignments_datasets = []

for i in range(len(aligned_datasets_GT)):
    good_alignments = []
    bad_alignments = []
    for j in range(len(aligned_datasets_GT[i])):
        alignment_validation = verify_alignment(aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j])
        if alignment_validation == True:
            good_alignments.append([aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j]])
        if alignment_validation == False:
            bad_alignments.append([aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j]])
    good_alignments_datasets.append(good_alignments)
    bad_alignments_datasets.append(bad_alignments)

for i in range(len(good_alignments_datasets)):
    print('Dataset:', i+1)
    print('Good alignments:', len(good_alignments_datasets[i]))
    print('Bad alignments:', len(bad_alignments_datasets[i]))



Dataset: 1
Good alignments: 19
Bad alignments: 6
Dataset: 2
Good alignments: 22
Bad alignments: 3
Dataset: 3
Good alignments: 24
Bad alignments: 1
Dataset: 4
Good alignments: 23
Bad alignments: 2


In [13]:
#for i in range(len(good_alignments_datasets)):
    #print(f'DATASET {i+1}:')
    #print('GOOD ALIGNMENTS:')
    #for j in range(len(good_alignments_datasets[i])):
        #print('GT:', good_alignments_datasets[i][j][0])
        #print('OCR:', good_alignments_datasets[i][j][1])
    #print('BAD ALIGNMENTS:')
    #for j in range(len(bad_alignments_datasets[i])):
        #print('GT:', bad_alignments_datasets[i][j][0])
        #print('OCR:', bad_alignments_datasets[i][j][1])

In [14]:
#d = {'aligned gt': [1, 2], 'aligned ocr': [3, 4], 'automatic validation': [5,6], 'manual validation': [7,8], 'dataset': [9,10]}
#alignment_df = pd.DataFrame(data=d)
alignment_df = pd.DataFrame(columns=['aligned gt', 'aligned ocr', 'automatic validation', 'manual validation', 'dataset'])

for i in range(len(good_alignments_datasets)): # for every dataset
    #print(f'DATASET {i+1}:')
    #print('GOOD ALIGNMENTS:')
    for j in range(len(good_alignments_datasets[i])):
        #print('GT:', good_alignments_datasets[i][j][0])
        #print('OCR:', good_alignments_datasets[i][j][1])
        alignment_df = alignment_df.append({'aligned gt': good_alignments_datasets[i][j][0], 'aligned ocr': good_alignments_datasets[i][j][1], 'automatic validation': True, 'manual validation': '-', 'dataset': i+1}, ignore_index=True)
    #print('BAD ALIGNMENTS:')
    for j in range(len(bad_alignments_datasets[i])):
        #print('GT:', bad_alignments_datasets[i][j][0])
        #print('OCR:', bad_alignments_datasets[i][j][1])
        alignment_df = alignment_df.append({'aligned gt': bad_alignments_datasets[i][j][0], 'aligned ocr': bad_alignments_datasets[i][j][1], 'automatic validation': False, 'manual validation': '-', 'dataset': i+1}, ignore_index=True)

alignment_df["automatic validation"].replace({True: "t", False: "f"}, inplace=True)

In [15]:
pd.set_option('display.max_columns', None)
alignment_df

,aligned gt,aligned ocr,automatic validation,manual validation,dataset
0,"[uyt, hamburgh, den, dito]","[uyt, hamburgh, den, dito]",t,-,1
1,"[wt, lisbon, van, den, selven, dito]","[wt, lisbon, van, den, selven, dito]",t,-,1
2,"[een, ander, uyt, prage, vanden, dito]","[een, ander, uyt, prage, vanden, dito]",t,-,1
3,[vranckryck],[vranckryck],t,-,1
4,"[wt, weenen, den, dito]","[wt, weenen, den, dito]",t,-,1
...,...,...,...,...,...
95,"[, de, regeering, van, valencia, heeft, in, ee...","[u, de, regeering, van, valencia, heeft, in, e...",t,-,4
96,"[verder, per, regel]","[verder, per, regel]",t,-,4
97,"[nieuwe, aanval, op, den, gulden, tegenspraak,...","[nieuwe, aanval, op, den, gulden, tegenspraak,...",t,-,4
98,"[formateurs, willen, nu, niet, langer, meer, w...","[formateurs, willen, nu, niet, langer, meer, ,...",f,-,4


In [16]:
for i in range(len(alignment_df)):
    print('GT:', alignment_df['aligned gt'][i])
    print('OCR:', alignment_df['aligned ocr'][i])
    try:
        for j in range(len(max([alignment_df['aligned gt'][i], alignment_df['aligned ocr'][i]], key=len))):
            print('i:', alignment_df['aligned gt'][i][j], alignment_df['aligned ocr'][i][j])
    except:
        pass
    print('Manual validation:', alignment_df['manual validation'][i])
    # fill in manual validation for given sentence
    while True:
        user_input = input("Manual validation: t/f")
        if user_input == 't' or user_input == 'f':
            alignment_df['manual validation'][i] = user_input
            break
        else:
            pass


GT: ['uyt', 'hamburgh', 'den', 'dito']
OCR: ['uyt', 'hamburgh', 'den', 'dito']
i: uyt uyt
i: hamburgh hamburgh
i: den den
i: dito dito
Manual validation: -
Manual validation: t/ft
GT: ['wt', 'lisbon', 'van', 'den', 'selven', 'dito']
OCR: ['wt', 'lisbon', 'van', 'den', 'selven', 'dito']
i: wt wt
i: lisbon lisbon
i: van van
i: den den
i: selven selven
i: dito dito
Manual validation: -
Manual validation: t/ft
GT: ['een', 'ander', 'uyt', 'prage', 'vanden', 'dito']
OCR: ['een', 'ander', 'uyt', 'prage', 'vanden', 'dito']
i: een een
i: ander ander
i: uyt uyt
i: prage prage
i: vanden vanden
i: dito dito
Manual validation: -
Manual validation: t/ft
GT: ['vranckryck']
OCR: ['vranckryck']
i: vranckryck vranckryck
Manual validation: -
Manual validation: t/ft
GT: ['wt', 'weenen', 'den', 'dito']
OCR: ['wt', 'weenen', 'den', 'dito']
i: wt wt
i: weenen weenen
i: den den
i: dito dito
Manual validation: -
Manual validation: t/ft
GT: ['wt', 'het', 'graefschap', 'benthem', 'den', 'janu']
OCR: ['wt', 'het'

In [22]:
alignment_df['dataset'].replace({1: "1600s", 2: "1700s", 3: "1800s", 4: "1900s",  }, inplace=True)
alignment_df


,aligned gt,aligned ocr,automatic validation,manual validation,dataset
0,"[uyt, hamburgh, den, dito]","[uyt, hamburgh, den, dito]",t,t,1600s
1,"[wt, lisbon, van, den, selven, dito]","[wt, lisbon, van, den, selven, dito]",t,t,1600s
2,"[een, ander, uyt, prage, vanden, dito]","[een, ander, uyt, prage, vanden, dito]",t,t,1600s
3,[vranckryck],[vranckryck],t,t,1600s
4,"[wt, weenen, den, dito]","[wt, weenen, den, dito]",t,t,1600s
...,...,...,...,...,...
95,"[, de, regeering, van, valencia, heeft, in, ee...","[u, de, regeering, van, valencia, heeft, in, e...",t,t,1900s
96,"[verder, per, regel]","[verder, per, regel]",t,t,1900s
97,"[nieuwe, aanval, op, den, gulden, tegenspraak,...","[nieuwe, aanval, op, den, gulden, tegenspraak,...",t,t,1900s
98,"[formateurs, willen, nu, niet, langer, meer, w...","[formateurs, willen, nu, niet, langer, meer, ,...",f,f,1900s


In [24]:
#alignment_df.to_csv('alignment_checker_df.csv')

In [4]:
alignment_checker_df = pd.read_csv('alignment_checker_df.csv')

In [23]:
TP = 0 # if manual validation is True   and automatic valudation is True
FP = 0 # if manual validation is False  and automatic valudation is True
TN = 0 # if manual validation is False  and automatic valudation is False
FN = 0 # if manual validation is True   and automatic valudation is False

for index, row in alignment_df.iterrows(): 
    if row['manual validation'] == 't' and row['automatic validation'] == 't':
        TP += 1
    if row['manual validation'] == 'f' and row['automatic validation'] == 't':
        FP += 1
    if row['manual validation'] == 'f' and row['automatic validation'] == 'f':
        TN += 1
    if row['manual validation'] == 't' and row['automatic validation'] == 'f':
        FN += 1

# calculate precision, recall, F1.
print('TP', TP)
print('FP', FP)
print('TN', TN)
print('FN', FN)

accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2*((precision*recall)/(precision+recall))

print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('F1:', F1)



TP 84
FP 4
TN 11
FN 1
accuracy: 0.95
precision: 0.9545454545454546
recall: 0.9882352941176471
F1: 0.9710982658959537


In [15]:
def calc_scores(df):
    
    TP = 0 # if manual validation is True   and automatic valudation is True
    FP = 0 # if manual validation is False  and automatic valudation is True
    TN = 0 # if manual validation is False  and automatic valudation is False
    FN = 0 # if manual validation is True   and automatic valudation is False

    for index, row in df.iterrows(): 
        if row['manual validation'] == 't' and row['automatic validation'] == 't':
            TP += 1
        if row['manual validation'] == 'f' and row['automatic validation'] == 't':
            FP += 1
        if row['manual validation'] == 'f' and row['automatic validation'] == 'f':
            TN += 1
        if row['manual validation'] == 't' and row['automatic validation'] == 'f':
            FN += 1

    # calculate precision, recall, F1.
    print('TP', TP)
    print('FP', FP)
    print('TN', TN)
    print('FN', FN)

    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2*((precision*recall)/(precision+recall))

    #print('accuracy:', accuracy)
    #print('precision:', precision)
    #print('recall:', recall)
    #print('F1:', F1)
    
    return accuracy, precision, recall, F1

for century in ['1600s', '1700s', '1800s', '1900s']:    
    df = alignment_checker_df[alignment_checker_df['dataset'] == century]
    print(f'score alignment checker {century}:')
    accuracy, precision, recall, F1 = calc_scores(df)
    print('accuracy:', accuracy)
    print('precision:', precision)
    print('recall:', recall)
    print('F1:', F1)

score alignment checker 1600s:
TP 18
FP 1
TN 5
FN 1
accuracy: 0.92
precision: 0.9473684210526315
recall: 0.9473684210526315
F1: 0.9473684210526315
score alignment checker 1700s:
TP 20
FP 2
TN 3
FN 0
accuracy: 0.92
precision: 0.9090909090909091
recall: 1.0
F1: 0.9523809523809523
score alignment checker 1800s:
TP 23
FP 1
TN 1
FN 0
accuracy: 0.96
precision: 0.9583333333333334
recall: 1.0
F1: 0.9787234042553191
score alignment checker 1900s:
TP 23
FP 0
TN 2
FN 0
accuracy: 1.0
precision: 1.0
recall: 1.0
F1: 1.0


test


In [20]:

#import pickle
#with open("aligned_GT_test.txt", "wb") as fp:   #Pickling
#    pickle.dump(aligned_datasets_GT, fp)
    
#with open("aligned_OCR_test.txt", "wb") as fp:   #Pickling
#    pickle.dump(aligned_datasets_OCR, fp)

In [21]:
#import pickle
#with open("GT_sentences_datasets_test.txt", "wb") as fp:   #Pickling
#    pickle.dump(GT_sentences_datasets, fp)

#with open("OCR_sentences_datasets_test.txt", "wb") as fp:   #Pickling
#    pickle.dump(OCR_sentences_datasets, fp)
    